In [7]:
pip install psutil

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (precision_score, recall_score, f1_score,
                             roc_auc_score, average_precision_score)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import time
import sys
import psutil
import os

In [83]:
df = pd.read_csv("C:/Users/Absolomjr/Desktop/final year project dataset/ML-Network Intrustion Detection System/data/processed/dataset_selection.csv")  # after preprocessing


In [84]:
df.head()

,Source_IP,Destination_IP,Port,Payload_Size,Intrusion,Hour,Minute,Day,Weekday,Request_Type_FTP,...,Protocol_TCP,Protocol_UDP,User_Agent_Nikto/2.1.6,User_Agent_Wget/1.20.3,User_Agent_curl/7.68.0,User_Agent_nmap/7.80,User_Agent_python-requests/2.25.1,Status_Success,Scan_Type_Normal,Scan_Type_PortScan
0,23662783,83989990,167,2954.0,0,6,4,6,4,False,...,False,False,False,False,False,False,False,True,True,False
1,68429848,38936310,80,206.0,0,15,20,4,2,True,...,False,True,False,True,False,False,False,True,True,False
2,34256933,9068034,21,977.0,0,7,23,3,1,False,...,False,False,False,False,False,True,False,True,True,False
3,38582138,47389783,22,521.0,0,11,35,3,1,False,...,True,False,False,False,True,False,False,True,True,False
4,90073913,63106198,53,251.0,0,6,9,5,3,False,...,True,False,False,True,False,False,False,True,True,False


In [ ]:
# selecting features

# selected_features = [
#     'Scan_Type_Normal',
#     'Status_Success',
#     'Payload_Size',
#     'Scan_Type_PortScan',
#     'Port'
# ]


In [73]:
print(df.columns.tolist())


['Source_IP', 'Destination_IP', 'Port', 'Payload_Size', 'Intrusion', 'Hour', 'Minute', 'Day', 'Weekday', 'Request_Type_FTP', 'Request_Type_HTTP', 'Request_Type_HTTPS', 'Request_Type_SMTP', 'Request_Type_SSH', 'Request_Type_Telnet', 'Protocol_TCP', 'Protocol_UDP', 'User_Agent_Nikto/2.1.6', 'User_Agent_Wget/1.20.3', 'User_Agent_curl/7.68.0', 'User_Agent_nmap/7.80', 'User_Agent_python-requests/2.25.1', 'Status_Success', 'Scan_Type_Normal', 'Scan_Type_PortScan']


In [86]:
X = df.drop('Intrusion', axis=1)
y = df['Intrusion']


In [ ]:
# # Identify numeric columns only
# numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

In [ ]:
# # Convert datetime columns to Unix time
# for col in X.columns:
#     try:
#         X[col] = pd.to_datetime(X[col])
#         X[col] = X[col].astype("int64") // 10**9
#     except:
#         pass


C:\Users\Absolomjr\AppData\Local\Temp\ipykernel_16960\762730274.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  X[col] = pd.to_datetime(X[col])
C:\Users\Absolomjr\AppData\Local\Temp\ipykernel_16960\762730274.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  X[col] = pd.to_datetime(X[col])
C:\Users\Absolomjr\AppData\Local\Temp\ipykernel_16960\762730274.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.to_datetime(X[col])
C:\Users\Absolomjr\AppData\Local

In [ ]:
# # Convert IP address columns to integer form
# def convert_ip_to_int(ip_string):
#     try:
#         return int(ipaddress.ip_address(ip_string))
#     except:
#         return np.nan  # if not a valid IP

# for col in X.columns:
#     if X[col].dtype == object:  # possible IP or MAC or text
#         # detect if column looks like IP addresses
#         sample = X[col].dropna().astype(str).head(5).tolist()
#         if all("." in v and len(v.split(".")) == 4 for v in sample):
#             X[col] = X[col].apply(convert_ip_to_int)

In [ ]:
# # After conversions: keeping only numeric columns
# numeric_cols = X.select_dtypes(include=[np.number]).columns
# X = X[numeric_cols]

In [87]:
# Splitting  again after cleaning
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [88]:
from sklearn.impute import SimpleImputer

# Imputing the missing values before Scaling
numeric_imputer = SimpleImputer(strategy="median")
X_train[numeric_cols] = numeric_imputer.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = numeric_imputer.transform(X_test[numeric_cols])


In [89]:
# Scaling clean numeric columns safely

scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [90]:
#  Measure latency + compute metrics

def evaluate_model(model, X_test, y_test, name="Model"):
    start = time.perf_counter()
    preds = model.predict(X_test)
    latency_ms = (time.perf_counter() - start) * 1000

    return {
        "Model": name,
        "Precision": precision_score(y_test, preds, zero_division=0),
        "Recall": recall_score(y_test, preds, zero_division=0),
        "F1 Score": f1_score(y_test, preds, zero_division=0),
        "ROC-AUC": roc_auc_score(y_test, preds),
        "PR-AUC": average_precision_score(y_test, preds),
        "Latency (ms)": round(latency_ms, 4),
    }


In [91]:
#  Memory usage before/after training

def memory_usage_mb():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)

In [92]:
# TRAIN RANDOM FOREST (Known Threats)

rf_mem_before = memory_usage_mb()

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    n_jobs=-1,
    random_state=42
)

rf.fit(X_train, y_train)

rf_mem_after = memory_usage_mb()
rf_mem_used = rf_mem_after - rf_mem_before

rf_results = evaluate_model(
    rf, X_test, y_test, name="Random Forest"
)
rf_results["Memory (MB)"] = round(rf_mem_used, 4)

In [93]:
# TRAIN LOGISTIC REGRESSION (SGDClassifier for Online Learning)

sgd_mem_before = memory_usage_mb()

sgd = SGDClassifier(
    loss="log_loss",            # logistic regression
    max_iter=1000,
    learning_rate="optimal",
    random_state=42
)

sgd.fit(X_train_scaled, y_train)

sgd_mem_after = memory_usage_mb()
sgd_mem_used = sgd_mem_after - sgd_mem_before

sgd_results = evaluate_model(
    sgd, X_test_scaled, y_test, name="Logistic Regression (SGD)"
)
sgd_results["Memory (MB)"] = round(sgd_mem_used, 4)

In [94]:
# Comparison Table

comparison_df = pd.DataFrame([rf_results, sgd_results])
comparison_df

,Model,Precision,Recall,F1 Score,ROC-AUC,PR-AUC,Latency (ms),Memory (MB)
0,Random Forest,1.0,1.0,1.0,1.0,1.00000,55.5468,3.0742
1,Logistic Regression (SGD),0.0,0.0,0.0,0.5,0.09752,2.3743,0.7695
